<a href="https://colab.research.google.com/github/janzika/MATH3261-5285/blob/main/finite_difference_methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finite Difference Methods

In this lab, we will look at numerical methods for solving a simple PDE: the 1D wave equation. We will also give a brief introduction to the python coding language and the Google colaboratory environment. By the end of this lab you will be able to

- write notes and mathematical equations in markdown
- write basic python code to manipulate matrices, define functions, and plot data
- discretize a PDE to form a matrix equation for the forward operator
- use a space-time diagram to examine the effect of intitial conditions and boundary conditions in the 1D wave equation


# Introducing Python

## What is Python?

In this course, we will use [Python](https://www.python.org/) to run fluid dynamics simulations, analyze output, and plot figures and movies. Python is free, flexible, easy to use, and has tons of online resources for beginners. If you have a question about Python, my default answer will be "have you Googled it?"

You do not need to be fluent in Python or any other programming language for this course. We will run Python from within these notebooks, and you will get lots of guidance. You are also encouraged to complete this [free online tutorial](https://www.codecademy.com/learn/learn-python) to familiarise yourself with Python syntax.

## Google Colaboratory

We will run the python labs in [Google Colaboratory](colab.research.google.com). Google Colab is a web-based computational environment in which you can read, write, and execute interactive *notebooks* like the one you are reading. The python code runs on a virtual machine in the cloud, so you don't need to install python on your local machine.

Before you begin the lab, you will need to [sign up](https://accounts.google.com/signup) for a free Google account. If you do not wish to sign up for a Google account, that's fine: you will still be able to read through the lab. You just won't be able to make edits or run any code.

### Colaboratory and Google Drive

If you have a Google account, you can mount your Google drive within the Colab environment. This is not required to run the lab (figures and movies will be saved to the Colab virtual machine and played in your browser). But if you would like to save output you can do so by navigating to `drive/'My Drive'`

```
from google.colab import drive
drive.mount('/content/drive')
!ls drive/'My Drive'
```

If you would like, you can save a copy of this notebook to your local machine or to your Google drive so you can save your output or see any notes you made within the notebook.

### Markdown

Within a notebook you can display text, mathematical notation, images, etc, using [Markdown](https://www.markdownguide.org/). For example:

#### World's most awesome equations:

- Newton's second law: $F = m \dot{v}$.
- Euler's equation: $e^{\mathrm{i} \pi} + 1 = 0$.
- Wave equation ("_gnarly dude!_"):
$$
\frac{\partial^2 f}{\partial t^2} = c^2 \frac{\partial^2 f}{\partial x^2}.
$$

You can also write `in-line code` or include code snippets like the one shown below:

```
# plot 2D colorplot
plt.subplot(1,2,1)
plt.pcolormesh(xx, yy, f)
plt.title('f(x,y)')
plt.xlabel('x')
plt.ylabel('y')
plt.axis('square')
```

If you double-click any cell, you can see the Markdown code used to create it.

-----------
**Exercise**

- Double click this text cell to make it editable. Add some notes, code snippets, or equations in the cell.

------------

Colab also allows you to use "cells" of python code that be executed in real time.

--------
**Exercise**

- Move down to the next cell and enter the following python code:

```
string = "Hello world!"
print(string)
```

- When you are finished, press **SHIFT + ENTER** to run the cell (or click the play button beside the cell).

--------------


In [ ]:
# This is a blank code cell. You can add your python code below and press SHIFT + ENTER to execute, or just click
# the play button beside the cell

string = "Hello world!"
print(string)


Congratulations! You are now a pythonista. Variables carry over from cell to cell, so you can build complex scripts by sequentially running cells one after another.

-------------
**Exercise**


- Now create a new cell by clicking on the "+ Code" button in the menu above. In the new cell, enter the following python code:

```
longer_string = string + " spam! Spam! SPAM!"
print(longer_string)
```

- Don't forget to **SHIFT + ENTER** to run the cell.

---------------

In [ ]:
longer_string = string + " spam! Spam! SPAM!"
print(longer_string)

## Libraries

Python makes extensive use of freely available, open source *libraries*, which contain tonnes of useful functions etc that you can make use of. The following python code calls two standard libraries: `numpy` (pronounced "numb pie"), which contains useful functions for carrying out numerics, and `matplotlib`, which allows us to plot data.

Since we will be using these libraries repeatedly, we will abbreviate their names to `np` and `plt`, respectively.

Finally, the last line instructs `matplotlib` to plot figures in the Jupyter notebook, just below the cell that calls it. That way, you will be able to view and save figures within the notebook.

In [ ]:
# Numerics
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# plot figures in Jupyter notebook
%matplotlib inline

## Variables, arrays, and indexing

The first thing we need to do is define a numerical grid. We will do this using the `numpy` functions `linspace` and `meshgrid`. Thus, when we call them, we will use the alias we defined for this library, `np`, followed by a period `.` followed by the particular function we want from the library, e.g. `np.linspace`.

First, we create a vector `xv` of `2n+1` equally spaced gridpoints between $x = -1$ and $x = 1$. Again, press **shift + enter** to run the following cell.

In [ ]:
# Grid
n = 50
xv = np.linspace(-1,1,2*n+1)

print(xv.shape)
print(xv[0:10])
print(xv[-10:])

The command `print(xv.shape)` prints the dimensions of the array `xv`. In this case, it is a 1D array with `2n+1` elements, so the dimensions are $(2n+1,)$. Notice that the second dimension is simply blank, rather than `1`.

The commands `print(xv[0:10])` and `print(xv[-10:])` print the first and last ten elements of the array. Indexing in python is indicated using square brackets `[...]` and is referenced to the first element (for positive indices) or the last element (for negative indices). Thus, `xv[0:10]` can be read as: "first + 0 elements to first + 10 elements", and `xv[-10:]` can be read as: "last - 10 elements to last element".

Next we will define 2D arrays `xx` and `yy` using the function `meshgrid`.

In [ ]:
xx,yy = np.meshgrid(xv,xv,indexing='ij')

print(xx.shape)
print(yy.shape)

print(xx[0:10,0:10])
print(yy[0:10,0:10])

The array `xx` is an $2n+1 \times 2n+1$ matrix in which each row is the corresponding value of `xv`. Likewise, the array `yy` is an $2n+1 \times 2n+1$ matrix with each column having the corresponding value of `xv`.

Let's try plotting a simple function of x and y using the arrays `xx` and `yy`. The function we will use is

$$
f = \sin  (\pi x)
$$

Both the sine function and the constant pi are part of the `numpy` library, so we call the using `np.sin` and `np.pi`, respectively.

In [ ]:
# define function
f = np.sin(np.pi*xx)*np.cos(np.pi*yy)

## Plotting

We will make a 2D colorplot of this function using `plt.pcolormesh`, as well as a 1D plot of a slice of the function using `plt.plot`. We will also use some other functions from `matplotlib` to control the appearance of the figure, such as `subplot`, `title`, `xlabel`, etc.

In [ ]:
fig = plt.figure(figsize=(16,16))

# plot 2D colorplot
ax = fig.add_subplot(1,2,1)
plt.subplot(1,2,1)
plt.pcolormesh(xx, yy, f)
plt.title('f(x,y)')
plt.xlabel('x')
plt.ylabel('y')
plt.axis('square')

# plot 1D slice along y = 0
plt.subplot(1,2,2)
plt.plot(xv,f[:,n])
plt.title('f(x,0)')
plt.xlabel('x')
plt.axis('square')
plt.ylim(-1.1,1.1)

## Functions

A more flexible way to define a function is using the `def` command

```
# define function
def trigfun(a,x):
    return np.sin(a*x)

f = trigfun(np.pi,xx)
```

Notice the colon (`:`) after the `def` command and the indentation of the subsequent code. These have meaning in python. The colon tells python that the indented code is part of the definition. There is no `end` command in python. The next line has no indentation, so it is not part of the function definition.

--------------

**Exercise**

- Replace the function `f = np.sin(np.pi*xx)` with the function definition in the code snippet above. Replot the figure using the new `trigfun` function.

- Modify the function `trigfun` above so that it is $f = \cos (\pi y)$ and plot a 2D colormap of the function. To do this, you will need to run (**shift + enter**) both cells: the cell that defines `trigfun` and the cell that plots `f`.

- How should you change the 1D plot so that it shows a slice along $x = 0$ instead of $y = 0$?

- Try plotting a more complication function like $f = \sin (\pi x) \cos (\pi y$).

------------------

# Background

In this lab we will look at numerical methods for solving the unforced 1D wave equation:

$$
\frac{\partial u}{\partial t} + a \frac{\partial u}{\partial x} = 0, \qquad 0 \le x \le L, \quad 0 \le t \le T.
$$

Here, $u(x,t)$ represents a perurbation (for example, a wave height) at position $x$ and time $t$ on the domain $[0,L] \times [0, T]$. The boundary and initial conditions for the problem are, respectively,

$$
\begin{align}
u(0,t) = b(t), \quad & \mbox{boundary condition},\\
u(x,0) = c(x), \quad & \mbox{initial condition}.
\end{align}
$$

### Discretizing the field

To solve the wave equation, we are going to replace the fields and derivatives with discrete approximations. We begin by dividing up the $x$-axis into $N+1$ increments and the $t$-axis into $K+1$ increments:

$$
\begin{align}
x_n = n \Delta x, \quad & n = 0, 1, \cdots, N-1, N\\
t_k = k \Delta k, \quad & k = 0, 1, \cdots, K-1, K
\end{align}
$$

Note that $x_0 = 0$, $x_N = L$, $t_0 = 0$, and $t_K = T$.

We can then evaluate the continuous field $u(x,t)$ at these discrete points in space and time:

$$
u_n^k \equiv u(x_n, t_k) = u (n \Delta x, k \Delta t).
$$

We will use subscripts for the space index $(n)$ and superscripts for the time index $(k)$.

### Finite difference approximation for the time derivative

Let us rewrite the PDE as

$$
\frac{\partial u}{\partial t} = r(x,t),
$$

where

$$
r(x,t) = - a \frac{\partial u}{\partial x}.
$$

The next step is to estimate the time derivative by replacing it with a linear approximation:

$$
\frac{\partial u}{\partial t} \approx \frac{u_n^{k+1} - u_n^{k}}{\Delta t}
$$

You can see as $\Delta t \rightarrow 0$, this approximation will become more and more accurate.

The partial derivative involves the field $u(x,t)$ evaluated at two timesteps: the *current* time-step $t_k$ and the future time-step $t_{k+1}$. We now need to decide what timestep to evaluate the right-hand side, $r(x,t)$. For example, we would evaluate $r(x,t)$ at the current timestep, in which case the PDE becomes

$$
\frac{u_n^{k+1} - u_n^k}{\Delta t} = r_n^k.
$$

This can then be solved for the perturbation at the next timestep:

$$
u_n^{k+1} = u_n^k + r_n^k \Delta t.
$$

Thus, given the current value of the field ($u_n^k$) and its derivatives ($r_n^k$), we can calculate the next value ($u_n^{k+1}$).

There are many choices for how to write this finite difference approximations, each with different advantages and disadvantages in terms of speed, accuracy, and stablity. The finite difference scheme written above is perhaps the simplest and is called a **forward difference**, and gives us the future timestep explicitly in terms of the current timestep.

Another possibility is a **backward difference** in which we evaluate the right-hand side at the future timestep:

$$
\frac{u_n^{k+1} - u_n^k}{\Delta t} = r_n^{k+1}.
$$

In this case, I can't solve for the future state explicity; the equation above represents an implicit equation for $u_n^{k+1}$. These methods tend to be more computationally intensive but are more numerically stable.

In this lab, we will use the forward difference scheme for the time derivative. You are encouraged to think about how things would change if you used the backward difference scheme instead.

### Finite difference approximation for the space derivative

Now we need to consider how to deal with the spatial derivatives. To do this, it is worth looking carefully at the equation we are solving:

$$
\frac{\partial u}{\partial t} = r(x,t) = - a \frac{\partial u}{\partial x}.
$$

This is a hyperbolic equation. For $a>0$, we expect to obtain waves travelling in the positive $x$-direction. This means that perturbations at a given $x$ will be affected by values of the field *to the left* of this location, or **upwind**. It makes sense then to approximate the spatial derivative as

$$
\frac{\partial u}{\partial x} \approx \frac{u_n^k - u_{n-1}^k}{\Delta x}.
$$

Again, we notice that this approximation becomes more accurate as $\Delta x \rightarrow 0$.

Higher-order schemes, and schemes involving both upwind and downwind information, may be appropriate depending on the particular PDE being solved.


### Matrix representation of the 1D wave equation

Gathering together these approximations, we can write the **forward difference upwind finite difference approximation** of the 1D wave equations as a recurrance relation

$$
u_n^{k+1} = u_n^{k} - \frac{a \Delta t}{\Delta x} \left( u_n^k - u_{n-1}^k \right).
$$

The quantity $C = a \Delta t / \Delta x$ is an important parameter called the *Courant number*. It is set by the choice of discretization through the size of the timestep $\Delta t$ and grid spacing $\Delta x$. Rewriting the recurrance relation then gives us

$$
u_n^{k+1} = C \; u_{n-1}^k + (1-C) \; u_n^k.
$$

In addition, we have the boundary and initial conditions. In discretized form, these are

$$
\begin{align}
u_0^k = b^k = b(k\Delta t), \quad & \mbox{boundary condition}, \\
u_n^0 = c_n = c(n\Delta x), \quad & \mbox{initial condition}. \\
\end{align}
$$

For consistency, we must have $b^0 = c_0$.

We are now ready to write out the finite-difference approximation of the PDE:

$$
\begin{align}
u_1^{k+1} & = C \; b^k + (1-C) \; u_1^k, \\
u_2^{k+1} & = C \; u_1^k + (1-C) \; u_2^k, \\
\vdots \\
u_{N-1}^{k+1} & = C \; u_{N-2}^k + (1-C) \; u_{N-1}^k.\\
u_N^{k+1} & = C \; u_{N-1}^k + (1-C) \; u_N^k.\\
\end{align}
$$

In matrix form this becomes

$$
\left( \begin{array}{c} u_1 \\ u_2 \\ \vdots \\ u_{N-1} \\ u_{N} \end{array} \right)^{k+1} = \left( \begin{array}{ccccccc} 1 - C &  &  &  &   \\ C & 1 - C & & & \\ & \ddots & \ddots & & & \\ & & C & 1 - C \\ & & & C &1-C
\end{array}\right)  \left( \begin{array}{c} u_1 \\ u_2 \\ \vdots \\ u_{N-1} \\ u_{N} \end{array} \right)^{k} + \left( \begin{array}{c} C b^k \\ 0 \\ \vdots \\ 0 \\0 \end{array} \right).
$$

In compact form this can be written as

$$
\mathbf{u}^{k+1} = \mathsf{A} \mathbf{u}^k + C \; \mathbf{b}^k.
$$

**Notes:**
- This matrix is in the form of a lower left triangular matrices with non-zero entries along two diagonals.
- The boundary conditions (which we have specified by $b^k$) appear as a new "forcing term" on the right-hand side that enforces the boundary condition at $x = 0$.
- Given initial conditions $\mathbf{u}^0 = \mathbf{c}$, the above equation can be solved iteratively for $\mathbf{u}^1$, $\mathbf{u}^2$, $\cdots$

- In the limit of $\Delta t \rightarrow 0$ or $a \rightarrow 0$ we have $C = 0$ and the matrix $\mathsf{A}$ becomes simply the identity. This means that the initial condition will remain unchanged in this limit.

# Experiment

## Problem parameters and discretization

Let's start by defining our problem parameters and discretizing the space and time axes.

In [ ]:
# Problem parameters
L = 0.5       # domain size
T = 4         # integration time
a = 0.1      # wave speed

# Solver parameters
N = 50        # number of gridcells
K = 400       # number of timesteps

In [ ]:
# spatial grid
xv = np.linspace(0,L,N+1)
dx = L/N
print('dx =',dx)

# time grid
tv = np.linspace(0,T,K+1)
dt = T/K
print('dt =',dt)

# meshgrid
xx,tt = np.meshgrid(xv,tv,indexing='ij')

## Matrix representation

Let's form the matrix above for the space and time discretization we have used. To do this, it will be convenient to define a function that generates a tridiagonal matrix, i.e. a matrix with non-zero values along the diagonal and the adjacent cells.

In [ ]:
# create forward operator
def tridiag(a, b, c, N):
    return np.diag(np.repeat(a,N-1), -1) + np.diag(np.repeat(b,N), 0) + np.diag(np.repeat(c,N-1), 1)

C = a*dt/dx
print('Courant number C =',C)

A = tridiag(C, 1-C, 0, N)
print('A=')
print(A[0:10,0:10])

## Boundary and initial conditions

Now let's specify boundary and initial conditions. For now, we'll keep the boundary condition to be simply $u(0,t)=0$. You can experiment with more complex boundary conditions later.

For an initial condition, we will use a Gaussian profile centred at $x = x_0$ with width $D$,

$$
u(x,0) = c(x) = \mathrm{exp} \left( -\frac{(x-x_0)^2}{D^2} \right).
$$

In [ ]:
# boundary condition
b = np.zeros(K+1)

# initial condition
c = np.zeros(N+1)
D = 0.1
x0 = L/2
c = np.exp(-(xv-x0)**2/D**2)

# set up the field
u = np.zeros((N+1,K+1))
u[:,0] = c
u[0,:] = b

## Stepping forward in time

We now write a simple `for` loop to iteratively update the field $\mathbf{u}^{k+1}$ given the previous timestep $\mathbf{u}^{k}$. We also need to enforce the boundary condition, which we do using an additional forcing term $C \mathbf{b}^k$.

This calculation involves $K+1$ separate multiplications of an $N \times N$ matrix with an $N \times 1$ vector. For reasonable values of $K$ and $N$ it should only take a few seconds.

In [ ]:
for k in range(K):
  u[1:,k+1] = u[1:,k+1] + np.matmul(A,u[1:,k])
  u[1,k+1] = u[1,k+1] + C*b[k+1]

## Visualising the output

Now we are ready to look at the evolution of the perturbation. We will do this in two ways: a space-time diagram of $u(x,t)$ and snapshots at different times $t$.

In [ ]:
fig = plt.figure(figsize=(16,16))

ax = fig.add_subplot(2,1,1)
plt.pcolormesh(xx,tt,u,cmap=plt.cm.coolwarm)
plt.title(r'Space-time plot of 1D wave equation',fontsize=16)
plt.xlabel(r'x',fontsize=16)
plt.ylabel(r't',fontsize=16)

ax = fig.add_subplot(2,1,2)
cmap = plt.get_cmap('viridis')
colors = [cmap(k) for k in np.linspace(0,1,K)]
for k in range(K):
    if k % np.floor((K+1)/10) == 0:
        plt.plot(xv,u[:,k],color=colors[k],label='t=%4.1f' %tv[k])

plt.title(r'Snapshots of perturbation u(x,t)',fontsize=16)
plt.xlabel(r'x',fontsize=16)
plt.ylabel(r'u(x,t)',fontsize=16)
plt.legend(loc='upper right').draw_frame(False)

-----------------
**Exercise**

As you carry out these experiments, make notes of your observations in the text box at the bottom of this notebook.  

- In the space time diagram the wave propagates towards the top-right corner of the figure. What is the slope of this line, and why?

- Try changing the wave speed `a` and rerunning the simulation. How does this change your plots? How does it change the Courant number $C$ and the forward operator $\mathsf{A}$? Are there values of $C$ that cause the simulation to break down?

- Notice that there is a small decrease in the height of the perturbation with time. This is due to *numerical diffusion* caused by the discretization we have chosen. Try changing the space and time discretizations by modifying $L$, $N$, $T$ and $K$. How does this change the simulation?

- The width of the initial perturbation is given by the parameter `D`. Try making this larger or smaller. How does this change the numerical diffusion?

- Repeat the experiment with zero initial condition but a time-dependent boundary condition. What is the result? What is the physical interpretation of this experiment?
-------------------

## Challenge exercises

Here are some harder exercises to make you think a bit more deeply about finite differencing and discretizations.

----------------
**Exercise**

- How would you change the model parameters and the forward operator to get waves travelling to the left instead of the right?

- Better accuracy can be achieved with a higher order estimate of the spatial derivatives. This can be done with a second order upwind scheme that uses 3 spatial points instead of 2 [Note: care needs to be taken near the boundary]
:

$$
u_n^{k+1} = u_n^{k} - \frac{a \Delta t}{2 \Delta x} \left( 3 u_n^k - 4 u_{n-1}^k  + u_{n-2}^k\right).
$$


- So far we have only considered a Dirichlet boundary conditions, e.g. $u(0,t) = 0$. How would you change this to a Neumann boundary condition instead? For example,
$$
\frac{\partial u}{\partial x} (0, t) = 0.
$$

- Using the full operator $\mathsf{A}$ is actually not very efficient because it involves the product of an $N \times N$ matrix with most entries equal to zero. A faster method would be to use a `for` loop to cycle through the non-zero components of the operator only. Write some python code to update the field using a `for` loop instead of a matrix multiplication.

----------------


# Notes

Add your own observations, calculations, and notes here.